In [1]:
import os
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_predict
from dask_searchcv import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
# Load constants
genes = ['TP53', ['KRAS', 'HRAS', 'NRAS']]

In [3]:
# Load training data
base_url = 'https://github.com/greenelab/pancancer/raw'
commit='9fd9afbecdbb4f855ecc85bb282fc59e75c7744f'

mut_file = '{}/{}/data/pancan_mutation_freeze.tsv.gz'.format(base_url, commit)
mut_burden_file = '{}/{}/data/mutation_burden_freeze.tsv'.format(base_url, commit)
sample_freeze_file = '{}/{}/data/sampleset_freeze.csv'.format(base_url, commit)

mutation_df = pd.read_table(mut_file, index_col=0)
sample_freeze = pd.read_table(sample_freeze_file, index_col=0)
mut_burden = pd.read_table(mut_burden_file)